<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Аннотация:-проект-для-«Викишоп»" data-toc-modified-id="Аннотация:-проект-для-«Викишоп»-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Аннотация: проект для «Викишоп»</a></span></li><li><span><a href="#Знакомство-с-данными" data-toc-modified-id="Знакомство-с-данными-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Знакомство с данными</a></span><ul class="toc-item"><li><span><a href="#Выводы:" data-toc-modified-id="Выводы:-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Выводы:</a></span></li></ul></li><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Предобработка данных</a></span><ul class="toc-item"><li><span><a href="#Выводы:" data-toc-modified-id="Выводы:-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Выводы:</a></span></li></ul></li><li><span><a href="#Обучение-моделей" data-toc-modified-id="Обучение-моделей-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Обучение моделей</a></span><ul class="toc-item"><li><span><a href="#Выводы:" data-toc-modified-id="Выводы:-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Выводы:</a></span></li></ul></li><li><span><a href="#Финальное-тестирование" data-toc-modified-id="Финальное-тестирование-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Финальное тестирование</a></span><ul class="toc-item"><li><span><a href="#Выводы:" data-toc-modified-id="Выводы:-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Выводы:</a></span></li></ul></li><li><span><a href="#Итоговые-выводы" data-toc-modified-id="Итоговые-выводы-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Итоговые выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

## Аннотация: проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

**Цель** проекта: на основе набора данных с разметкой о токсичности правок построить и обучить модель для классификации комментариев на позитивные и негативные. 

**Задачи** проекта: 
- обучить несколько моделей машинного обучения;
- обеспечить достижения метрикой качества *F1* значения не меньше 0.75.

## Знакомство с данными

In [96]:
#импортируем библиотеки и инструменты, которые понадобятся нам при выполнении проекта
import pandas as pd
import re 
import warnings
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier

In [97]:
#применяем специальный метод для чтения исходного файла
initial_data = pd.read_csv('/datasets/toxic_comments.csv')

#выводим на экран ключевые данные об исходном датасете
display(initial_data.head(10))
print('-'*25)
print(initial_data.shape)
print('-'*25)
print(initial_data.isnull().sum())
print('-'*25)
print('Количество явных дубликатов:', initial_data.duplicated().sum())
print('-'*25)
print(initial_data['toxic'].value_counts())

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
5,"""\n\nCongratulations from me as well, use the ...",0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0


-------------------------
(159571, 2)
-------------------------
text     0
toxic    0
dtype: int64
-------------------------
Количество явных дубликатов: 0
-------------------------
0    143346
1     16225
Name: toxic, dtype: int64


### Выводы:

Знакомство с исходными данными позволило выявить следующее:
- комментарии написаны на английском языке;
- тексты комментариев не очищены, часто встречается символ новой строки;
- в текстах использованы буквы различных регистров;
- пропусков и явных дубликатов не обнаружено;
- наблюдается дисбаланс классов целевого признака.

На следующем этапе обработаем тексты комментариев для улучшения их качества в целях машинного обучения.

## Предобработка данных

В рамках предобработки исходных данных очистим тексты комментариев от знаков новой строки, приведем все символы к нижнему регистру, оставим в них только символы латиницы и пробелы, произведем лемматизацию.

Начнем с очистки текстов, создадим специальную функцию, которая выполнит выше названные задачи:

In [98]:
warnings.filterwarnings('ignore') #отключим предупреждения на случай возникновения в моделях параметров, при которых 
                                  #не удастся вычислить F1-меру
        
def clear_text(text): #создадим функцию для очистки текстов комментариев
    text = text.lower()
    text = re.sub('\n', ' ', text)
    text = re.sub(r'[^a-zA-Z ]', ' ', text)
    return text

initial_data['new_text'] = initial_data['text'].apply(clear_text) #применим функцию к столбцу с текстами, создав новый 
                                                                  #столбец, чтобы была возможность вернуться к исходным
                                                                  #текстам и провести сравнение

Далее лемматизируем тексты. Так как комментарии оставлены на английском языке, то используемые нами ранее инструменты для лемматизации не подойдут. Используем WordNetLemmatizer из библиотеки NLTK:

In [99]:
def get_wordnet_pos(word): #создадим функцию для определения части речи слов из текстов
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()

def get_word_text (corpus): #создадим функцию для токенизации и лемматизации корпуса текстов
    lemmatized = ' '.join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(corpus)])
    return lemmatized

nltk.download('averaged_perceptron_tagger')

initial_data['new_text'] = initial_data['new_text'].apply(get_word_text) #применим функцию к очищенным комментариям

display(initial_data.head(10)) #выведем результат преобразований на экран
print()
print('Количество явных дубликатов:', initial_data.duplicated().sum()) #еще раз проверим на явные дубликаты

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


,text,toxic,new_text
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits make under my userna...
1,D'aww! He matches this background colour I'm s...,0,d aww he match this background colour i m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not try to edit war it s ju...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i can t make any real suggestion on impro...
4,"You, sir, are my hero. Any chance you remember...",0,you sir be my hero any chance you remember wha...
5,"""\n\nCongratulations from me as well, use the ...",0,congratulation from me a well use the tool wel...
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,cocksucker before you piss around on my work
7,Your vandalism to the Matt Shirvington article...,0,your vandalism to the matt shirvington article...
8,Sorry if the word 'nonsense' was offensive to ...,0,sorry if the word nonsense be offensive to you...
9,alignment on this subject and which are contra...,0,alignment on this subject and which be contrar...



Количество явных дубликатов: 0


Теперь разделим имеющиеся у нас данные на обучающую, валидационную и тестовую выборки в соотношении 3:1:1.

<div class="alert alert-block alert-success">
<b>Успех:</b> Очистка и лемматизация были сделаны верно.
</div>

In [100]:
#выделяем признаки и целевой признак
features = initial_data['new_text']
target = initial_data['toxic']

#делим данные на выборки
features, features_test, target, target_test = train_test_split(features, target, test_size = 0.2, random_state = 555)
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size = 0.25, random_state = 555)

print(features_train.shape, features_test.shape,  features_valid.shape) #проверяем оотношение

(95742,) (31915,) (31914,)


### Выводы:

На этапе предобработки данных была произведена очистка и лемматизация текстов комментариев, а также приведение всех символов в текстах к нижнему регистру. В результате исходный фрейм дополнился столбцом с текстами, готовыми для использования в качестве признаков при обучении, проверке и тестировании моделей машинного обучения. 

Кроме того в отдельные переменные были выделены признаки и целевой признак, а данные разделены на три выборки: обучающую, валидационную и тестовую. 

Можем переходить к обучению моделей.

## Обучение моделей

В данном проекте мы обучим две модели: логистическую регрессию и CatBoost. Т.к. нам нужно определить тональность текстов комментариев, то в качестве признаков будут использованы величины TF-IDF.

In [101]:
tf_idf = TfidfVectorizer() #вызываем счетчик

#определяем величины TF-IDF признаков в выборках
tf_idf_train = tf_idf.fit_transform(features_train) 
tf_idf_valid = tf_idf.transform(features_valid) 
tf_idf_test = tf_idf.transform(features_test) 

#обучаем модель логистической регрессии, балансируем классы
model = LogisticRegression (class_weight ='balanced', random_state = 555)
model.fit(tf_idf_train, target_train)
predictions = model.predict(tf_idf_valid)
f1_regress = f1_score(target_valid, predictions) 
print(f1_regress.round(2))

0.74


In [102]:
#обучаем модель CatBoost, параметры подбираем вручную
cat_model = CatBoostClassifier(eval_metric='F1', iterations = 1600)
cat_model.fit(tf_idf_train, target_train, verbose = 500)
pred_valid = cat_model.predict(tf_idf_valid) 
cat_result = f1_score(target_valid, pred_valid) 
print(cat_result.round(2))

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

Learning rate set to 0.046956
0:	learn: 0.4218737	total: 3.79s	remaining: 1h 40m 54s
500:	learn: 0.7463637	total: 27m 20s	remaining: 59m 59s
1000:	learn: 0.7925708	total: 53m 46s	remaining: 32m 10s
1500:	learn: 0.8166903	total: 1h 20m 31s	remaining: 5m 18s
1599:	learn: 0.8197069	total: 1h 26m 13s	remaining: 0us
0.77


### Выводы:

При проверке на валидационной выборке модель логистической регрессии не смогла преодолеть пороговое значение метрики F1. В то же время модель CatBoost показала лучший результат, достигнув значения 0.77. 

На следующем этапе произведем финальное тестирование моделей и выберем из них наиболее подходящую.

## Финальное тестирование

Получим предсказания обученных ранее моделей на тестовой выборке и подсчитаем значение метрики качества. Финальное тестирование будет проводиться в том же порядке, что и обучение.

In [103]:
predictions_test = model.predict(tf_idf_test)
f1_regress_test = f1_score(target_test, predictions_test) 
print('Результат логистической регрессии:', f1_regress_test.round(2))

Результат логистической регрессии: 0.75


In [104]:
pred_test = cat_model.predict(tf_idf_test)
cat_result_test = f1_score(target_test, pred_test) 
print('Результат CatBoost:', cat_result_test.round(2))

Результат CatBoost: 0.76


### Выводы:

Модель логистической регрессии на тестовой выборке показала результат лучше, чем на валидационной, однако даже при этом она смогла достичь лишь нижней границы допустимых значений метрики F1. 

Результат модели CatBoost наоборот снизился по сравнению с предыдущим этапом, при этом он все равно выше, чем у логистической регрессии.

## Итоговые выводы

В ходе выполения проекта по построению модели, классифицирующей комментарии на позитивные и негативные, были решены следующие задачи:
- произведена обработка текстов комментариев с целью улучшения их качества для дальнейшего использования в машинном обучении: очистка, токенизация, лемматизация;
- обучены две модели машинного обучения - линейная регрессия и СatBoost;
- произведена финальная проверка отобранных моделей на тестовой выборке;
- оценено качество полученных предсказаний.

В результате, искомой моделью стала модель CatBoost с количеством итераций, равным 1600. Значение метрики качества данной модели на тестовой выборке составило 0.76.